In [ ]:
# bibliotecas
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report
from tensorflow.keras.preprocessing.image import load_img, img_to_array


In [ ]:
# clonar repositório
!git clone https://github.com/PDI20/pdi_2024_2025.git

In [ ]:
# unzip do dataset
!unzip /content/pdi_2024_2025/dataset/flowers.zip

In [ ]:
input_size = (299, 299)

# carregar imagens
train_set = tf.keras.utils.image_dataset_from_directory(
    "/content/flowers/train/",
    seed=1337,
    image_size=input_size,
    label_mode='categorical'
)

val_set = tf.keras.utils.image_dataset_from_directory(
    "/content/flowers/valid/",
    seed=1337,
    image_size=input_size,
    label_mode='categorical'
)

test_set = tf.keras.utils.image_dataset_from_directory(
    "/content/flowers/test/",
    seed=1337,
    image_size=input_size,
    label_mode='categorical'
)

# classes
class_names = train_set.class_names

In [ ]:
examples_per_class = {class_name: [] for class_name in class_names}

# 5 imagens de cada classe
for images, labels in train_set:
    for img, label in zip(images, labels):
        class_idx = tf.argmax(label).numpy()  # Convert one-hot encoded label to class index
        class_name = class_names[class_idx]

        # Only add examples if we have fewer than 5 for this class
        if len(examples_per_class[class_name]) < 5:
            examples_per_class[class_name].append(img)

    # Stop if we have 5 examples for each class
    if all(len(img_list) == 5 for img_list in examples_per_class.values()):
        break

# Plot
fig, axs = plt.subplots(len(class_names), 5, figsize=(15, len(class_names) * 3))
fig.suptitle("Exemplos", fontsize=16)
for row, class_name in enumerate(class_names):
    for col in range(5):
        axs[row, col].imshow(examples_per_class[class_name][col].numpy())
        axs[row, col].axis("off")
        if col == 0:
            axs[row, col].set_title(class_name)

plt.tight_layout()
plt.subplots_adjust(top=0.95)
plt.show()

In [ ]:
# normalização das imagens
train_set = train_set.map(lambda x, y: (x / 255.0, y))
val_set = val_set.map(lambda x, y: (x / 255.0, y))
test_set = test_set.map(lambda x, y: (x / 255.0, y))

In [ ]:
# inicializar modelo
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# congelar todas as layers
for layer in base_model.layers:
    layer.trainable = False

# descongelar as últimas 50
for layer in base_model.layers[-50:]:
    layer.trainable = True

# adicionar uma nova layer de saída ao modelo, para classificar as classes corretas
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(2, activation='softmax')  # 2 classes
])

# compilar modelo
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Early stopping callback para parar o treino quando não existe melhoria da validation loss
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,               # pára o treino após 5 épocas
    restore_best_weights=True # reverter os pesos do treino para os melhores obtidos
)

# Reduce learning rate callback para atualizar o learning rate quando não existe melhoria da validation loss
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,               # reduzir learning rate por um fator de 0.1
    patience=3,               # atualiza após 3 épocas
    min_lr=1e-10              # learning rate mínimo
)

model_checkpoint = ModelCheckpoint('/content/model_checkpoint.keras', save_best_only=True, monitor='val_loss', mode='min'),

In [ ]:
# treinar o modelo
history = model.fit(train_set,
                    batch_size = 32,
                    epochs=100,
                    validation_data=val_set,
                    callbacks=[early_stopping, reduce_lr, model_checkpoint])

In [ ]:
# criar figura
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Accuracy plot
axes[0].plot(history.history['accuracy'], label='train_accuracy')
axes[0].plot(history.history['val_accuracy'], label='val_accuracy')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].set_title('Model Accuracy')

# Loss plot
axes[1].plot(history.history['loss'], label='train_loss')
axes[1].plot(history.history['val_loss'], label='val_loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].set_title('Model Loss')

plt.tight_layout()
plt.show()

In [ ]:
# avaliar treino com base nos dados de teste
test_loss, test_accuracy = model.evaluate(test_set)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

In [ ]:
def test_confusion_matrix(model, test_set, class_names):

    # labels e previsões
    y_true = []
    y_pred = []

    for images, labels in test_set:
        predictions = model.predict(images)
        y_pred.extend(np.argmax(predictions, axis=1))  # Previsão da classe


        if labels.shape[-1] > 1:
            y_true.extend(np.argmax(labels, axis=1))
        else:
            y_true.extend(labels.numpy())

    # lista para array
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Generate classification report
    print("\n Relatório de Classificação:")
    print(classification_report(y_true, y_pred, target_names=class_names))

    # matriz de confusão
    cm = confusion_matrix(y_true, y_pred)

    # matrix de confusão
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(cmap="Blues", xticks_rotation=45)
    plt.title("Matriz de Confusão")
    plt.show()

In [ ]:
test_confusion_matrix(model, test_set, class_names)

In [ ]:
image_path = ""

# Load the image and resize to the model's expected input size
img = load_img(image_path, target_size=target_size)

# Convert the image to an array and normalize (to range [0, 1] if needed)
img_array = img_to_array(img) / 255.0  # Normalize as done in training

# Add an extra batch dimension since the model expects a batch of images
img_array = np.expand_dims(img_array, axis=0)

# Predict using the model
predictions = model.predict(img_array)

# Get the predicted class index
predicted_class_idx = np.argmax(predictions, axis=1)[0]
predicted_class = class_names[predicted_class_idx]
confidence = predictions[0][predicted_class_idx]

# Print results
print(f"Predicted class: {predicted_class} with confidence: {confidence * 100:.2f}%")

In [ ]:
# inicializar modelo
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
base_model.trainable = False  # congelar o modelo

# adicionar uma nova layer de saída ao modelo, para classificar as classes corretas
model_load_weights = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(2, activation='softmax')  # 2 classes
])

# compilar modelo
model_load_weights.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_load_weights.build(input_shape=(None, 299, 299, 3))

# carregar pesos
model_load_weights.load_weights("/content/model_checkpoint.keras")

In [ ]:
# Define the path and input size
image_path = "/content/images_test/test.jpg"
input_size = (299, 299)  # Define the input size used during training

# Load the image and resize to the model's expected input size
img = load_img(image_path, target_size=input_size)

# Convert the image to an array and normalize (to range [0, 1] if needed)
img_array = img_to_array(img) / 255.0  # Normalize as done in training

# Add an extra batch dimension since the model expects a batch of images
img_array = np.expand_dims(img_array, axis=0)

# Predict using the model
predictions = model.predict(img_array)

# Get the predicted class index
predicted_class_idx = np.argmax(predictions, axis=1)[0]
predicted_class = class_names[predicted_class_idx]
confidence = predictions[0][predicted_class_idx]

# Print results
print(f"Predicted class: {predicted_class} with confidence: {confidence * 100:.2f}%")

# Display the image with prediction
plt.imshow(img)
plt.title(f"Predicted: {predicted_class} ({confidence * 100:.2f}%)")
plt.axis("off")  # Hide axes for a cleaner look
plt.show()
